<a href="https://colab.research.google.com/github/vlopezma/Se-ales-1/blob/main/Valeria_Lopez_Parcial1_SyS_2024II_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

Primero, definimos y graficamos la señal original x(t)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist


# Señal original:
t = np.linspace(0, 0.01, 10000) # rango de tiempo para evaluar la señal
xt = 0.3*np.cos(1000*np.pi*t - np.pi/4) + 0.6*np.sin(2000*np.pi*t) + 0.1*np.cos(11000*np.pi*t - np.pi)

# Graficar la señal x(t) continua
plt.figure(figsize=(10, 6))
plt.plot(t, xt, label='x(t)')
plt.xlabel('Tiempo (s)')
plt.ylabel("Amplitud")
plt.title('Señal continua')
plt.legend()
plt.grid(True)
plt.show()


La función my_ceropen se utiliza para normalizar la señal continua x(t) y ajustar sus valores al rango definido

In [ ]:
def my_ceropen(xt, ymin=-5, ymax=5):  # en general se pueden definir valores por defecto
    '''
    Código base para simular proceso de cero y pendiente.
    Se ingresa un arreglo de numpy y los valores min y max después de cero y pendiente.
    '''
    xmax = max(xt)  # x.max()
    xmin = min(xt)  # x.min()
    m = (ymax - ymin) / (xmax - xmin)
    c = ymin - m * xmin
    yv = m * xt + c
    return yv

ycs = my_ceropen(xt, ymin=-3.3, ymax=3.3)


In [ ]:
#cero y pendiente
ycs = my_ceropen(xt, ymin=-3.3,ymax=3.3)

# Parámetros, número de bits y vector de cuantización
nbits = 5
rmin = -3.3
rmax = 3.3
ve = np.linspace(rmin,rmax,2**nbits) # rango de tiempo para evaluar la señal

dn = cdist(ycs.reshape(-1,1),ve.reshape(-1,1))
#se requiere identificar el elemento ve[j] más cercano a y[i] para genera señal cuantizada
ind = np.argmin(dn,axis=1) #el parámetro axis = 1 indica que busca la posición a lo largo de las columnas del elemento más pequeño en cada fila ind

def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]


plt.plot(t,ycs,c='r',label='ydig')

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

 Función que permite muestrear la señal continua y representar gráficamente sus valores discretos en función del tiempo.

In [ ]:
def mg(fs, duracion=0.01, titulo=''):
    # Funcion que muestra y grafica la señal x(t) con la frecuencia de muestreo fs.

    Ts = 1/fs   # Periodo de muestreo
    t = np.arange(0, duracion, Ts)  # Vector de tiempo

    # Señal x(t)
    x_t = 0.3*np.cos(1000*np.pi*t - np.pi/4) + 0.6*np.sin(2000*np.pi*t) + 0.1*np.cos(11000*np.pi*t - np.pi)

    # Gráfica de la señal muestreada
    plt.figure(figsize=(10, 6))
    plt.stem(t, x_t, linefmt='b-', markerfmt="bo", basefmt="k")
    plt.title(titulo)
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud')
    plt.grid(True)
    plt.show()

In [ ]:
mg(5000, titulo='Señal discretizada x(t) a 5kHz')

De acuerdo con el teorema de Nyquist, la frecuencia de muestreo debe ser al menos el doble de la frecuencia máxima presente en la señal, para evitar aliasing. En este caso, la frecuencia de muestreo proporcionada (5kHz) no es adecuada, ya que es menor que Fmax = 22kHz. Por lo tanto, la señal no se puede reconstruir correctamente debido a la pérdida de información en el proceso de muestreo.

In [ ]:
mg(11000, titulo='Señal discretizada x(t) a 11kHz')

# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
from P1_model import system_ # Importamos el sistema modelado como una "caja negra"

my_system = system_.My_System() # Instanciamos el sistema
my_system.create_()  # Inicializamos el sistema
fs = my_system.fs  # Recuperamos la frecuencia de muestreo del sistema

# Definición de la señal de entrada
t = np.arange(-0.01, 0.02, 1/fs)  # vector de tiempo desde -0.01 s a 0.02 s con pasos de 1/fs
signal_u = np.heaviside(t,1) # # Generamos una señal de tipo escalón unitario (Heaviside) como entrada
y_u = my_system.predict(signal_u) # El sistema procesa la señal de entrada y genera la señal de salida

# Visualización de las señales
fig, axs = plt.subplots(2,1) # dos subgráficos: uno para la entrada y otro para la salida
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

Analizamos la respuesta del sistema a un escalón unitario, procesando la entrada y graficando la salida para observar su comportamiento. demás, evaluamos la linealidad del sistema, verificando si cumple la propiedad:
$$
S({ax_1[n]}+bx_2[n])=aS(x_1)+bS(x_2)
$$


In [ ]:
# Escalar la entrada por un factor a
a = 2
u_scaled = a * signal_u #señal original por el factor el factor a escalar

# Salida del sistema para a*u(t)
y2 = my_system.predict(u_scaled)

# Escalar la salida original
y1_scaled = a * y_u

# Comparar gráficamente
plt.figure(figsize=(10, 6))
plt.plot(t, y2, label="Salida del sistema para a*u(t) (y2)", linestyle="--")
plt.legend()
plt.title("Verificación de Linealidad: Homogeneidad")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.show()
plt.figure(figsize=(10, 6))
plt.plot(t, y1_scaled, label="a * Salida original (y1_scaled)")
plt.legend()
plt.title("Verificación de Linealidad: Homogeneidad")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.show()

# Error numérico
error = np.linalg.norm(y2 - y1_scaled)
print(f"Error entre y2 y y1_scaled: {error}")

Verificamos la linealidad (homogeneidad) del sistema comparando la salida al escalar la entrada por 𝑎=2 (𝑦2) con la salida original escalada por el mismo factor (y1scaled​). Ambas salidas se grafican para compararlas, y se calcula el error numérico entre ellas. Si el error es pequeño, el sistema cumple con la propiedad de homogeneidad.

In [ ]:
k = 0.005  # Tiempo de desplazamiento (k)

# Generar la señal de entrada desplazada x[n-k]
x = np.heaviside(t - k, 1)  # Función escalón unitario desplazada en k

# Obtener la salida del sistema para la señal desplazada x[n-k]
y = my_system.predict(x)  # Calculamos la salida del sistema con la entrada desplazada

# Comparar gráficamente
plt.figure(figsize=(12, 6))  # Configuramos el tamaño de la figura

# Graficar la entrada desplazada x[n-k]
plt.subplot(2, 1, 1)  # Primer gráfico (arriba)
plt.plot(t, x, label="Entrada desplazada x[n-k]")  # Señal de entrada desplazada
plt.grid()  # Mostramos la cuadrícula
plt.legend()  # Mostramos la leyenda
plt.title("Entrada desplazada del sistema (x[n-k])")  # Título del gráfico

# Graficar la salida del sistema para x[n-k]
plt.subplot(2, 1, 2)  # Segundo gráfico (abajo)
plt.plot(t, y, label="Salida desplazada y[n-k]", linestyle="--")  # Salida correspondiente
plt.legend()  # Mostramos la leyenda
plt.title("Salida del sistema para entrada desplazada (y[n-k])")  # Título del gráfico

# Ajustes y visualización
plt.xlabel("Tiempo [s]")  # Etiqueta del eje X
plt.grid()  # Mostramos la cuadrícula
plt.tight_layout()  # Ajustamos los espacios entre gráficos
plt.show()  # Mostramos la figura

# Comparación numérica de las salidas (deshabilitada en el código actual)
# error = np.linalg.norm(y - yu1)  # Calcula el error entre salidas si es necesario
# print(f"Error entre las salidas: {error}")  # Muestra el error numérico si está habilitado


Verificamos la invarianza en el tiempo del sistema al desplazar la entrada escalón unitario en 𝑘=0.005 segundos. Se calcula la salida para esta entrada desplazada y se grafican tanto la entrada como la salida para analizar si el sistema responde únicamente con un desplazamiento temporal, manteniendo la forma de la señal.

In [ ]:
g = np.diff(y_u)  # Calcula las diferencias entre valores consecutivos de la salida 'y_u', representando su derivada discreta.

min_len = min(len(t), len(g))  # Encuentra el tamaño mínimo entre los vectores de tiempo 't' y las diferencias 'g'.

t = t[:min_len]  # Ajusta el tamaño del vector de tiempo 't' para que coincida con el tamaño de 'g'.
x1 = x[:min_len]  # Ajusta el tamaño del vector 'x1' para que coincida con 'g' (necesario si se grafica contra 'g').

# Graficar la señal g en función del tiempo t
plt.stem(t, g, label='x1')  # Grafica la derivada discreta de 'y_u' (g) como un gráfico de líneas con marcadores.
plt.legend()  # Agrega una leyenda al gráfico.
plt.grid()  # Activa la cuadrícula para facilitar la lectura de los datos.
plt.xlabel('t')  # Etiqueta el eje X como 't' (tiempo).
plt.show()  # Muestra el gráfico.


In [ ]:
n = t  # Asignamos el vector de tiempo 't' al nuevo vector 'n'.
print(len(n))  # Imprimimos la longitud del vector 'n' para verificar su tamaño.

# Generamos la señal 'xn' como la suma de dos ondas seno con diferentes frecuencias.
xn = np.sin(100 * np.pi * n) + np.sin(600 * np.pi * n)

# Calculamos la convolución de las señales 'g' (derivada discreta anterior) y 'xn'.
con = np.convolve(g, xn)

# Aseguramos que 'n' y 'con' tengan el mismo tamaño, tomando el tamaño mínimo.
min_len = min(len(n), len(con))
n = n[:min_len]  # Ajustamos 'n' al tamaño mínimo.
con = con[:min_len]  # Ajustamos 'con' al tamaño mínimo.

# Graficamos la señal convolucionada 'con' contra el tiempo 'n'.
plt.stem(n, con, label='x1')  # Usamos un gráfico de líneas con marcadores para 'con'.
plt.legend()  # Agregamos una leyenda al gráfico.
plt.grid()  # Mostramos la cuadrícula para facilitar la lectura.
plt.xlabel('t')  # Etiquetamos el eje X como 't' (tiempo).
plt.show()  # Mostramos el gráfico.


In [ ]:
def convgraf(h, x):
    """
    Función para calcular y visualizar el proceso de convolución paso a paso.

    Parámetros:
    - h: Vector del filtro (o señal desplazada).
    - x: Vector de la señal de entrada.

    Retorna:
    - xm: Matriz con los reflejos y desplazamientos de x.
    - ym: Matriz con los resultados parciales de la convolución.
    - hm: Versión extendida del filtro h.
    """
    lx = len(x)  # Longitud de la señal de entrada x
    lh = len(h)  # Longitud del filtro h
    M = 2 * lx + lh  # Tamaño máximo necesario para visualizar el proceso completo

    # Inicializamos las matrices para reflejos, desplazamientos y acumulaciones
    xm = np.zeros((M, M))  # Matriz para reflejos y desplazamientos de x
    hm = np.r_[np.zeros((lx, 1)), h, np.zeros((lx, 1))]  # Extensión del filtro h con ceros
    ym = np.zeros((M, M))  # Matriz para resultados parciales de convolución

    # Proceso de convolución paso a paso
    for i in range(M - lx + 1):  # Itera por todos los desplazamientos posibles
        xm[i:i+lx, i] = np.flip(x).reshape(-1)  # Refleja (flip) y desplaza la señal de entrada x
        ym[i, i] = xm[:, i].T.dot(hm)  # Calcula el producto punto entre x reflejada y h

    return xm, ym, hm  # Devuelve las matrices para análisis y graficación

def plot_conv(k, xm, ym, hm):
    """
    Función para graficar un paso del proceso de convolución.

    Parámetros:
    - k: Paso actual de la convolución.
    - xm: Matriz con reflejos y desplazamientos de x.
    - ym: Matriz con los resultados parciales.
    - hm: Versión extendida del filtro h.
    """
    plt.stem(xm[:, k], markerfmt='+', label='$x[k]$')  # Gráfica de x reflejada y desplazada en el paso k
    plt.stem(hm, linefmt='g', markerfmt='.', label='$h[n-k]$')  # Gráfica del filtro h desplazado
    plt.stem(ym[:k, :].sum(axis=0), markerfmt='s', linefmt='r',
             label='$y[n]=\sum^{\infty}_{k=-\infty}x[k]h[n-k]$')  # Salida acumulada hasta el paso k
    plt.legend()  # Añade leyendas al gráfico
    plt.show()  # Muestra la gráfica
    return

# Reformateo de las señales de entrada y filtro
g1 = g.reshape(-1, 1)  # Convierte la señal g en un vector columna
x1n = xn.reshape(-1, 1)  # Convierte la señal xn en un vector columna

# Cálculo del proceso de convolución
xm, ym, hm = convgraf(x1n, g1)  # Calcula las matrices para visualizar la convolución

# Ejemplo de graficación de un paso específico
k = 10  # Escoge un paso específico para graficar
plot_conv(k, xm, ym, hm)  # Muestra el gráfico interactivo para el paso k


In [ ]:
# Función interactiva para explorar el proceso de convolución
@interact(k=(0, xm.shape[0] - len(x1n), 1))  # Control deslizante para elegir el paso k
def show_frame(k=0):  # Función que grafica el paso actual de convolución
  plot_conv(k, xm, ym, hm)  # Llama a la función de graficación para el paso seleccionado

plt.show()

Con base en las simulaciones realizadas, se concluye que el sistema cumple con las propiedades de linealidad e invarianza en el tiempo, lo que permite clasificarlo como un sistema SLIT (Sistema Lineal e Invariante en el Tiempo). Esto se verificó mediante pruebas de homogeneidad, aditividad y desplazamiento temporal, las cuales confirmaron que el sistema responde de manera consistente y proporcional a las señales de entrada. Además, se utilizó su respuesta al impulso para calcular correctamente la salida ante la señal compuesta
𝑥
[
𝑛
]
=
sin
⁡
(
100
𝜋
𝑛
)
+
sin
⁡
(
600
𝜋
𝑛
)
x[n]=sin(100πn)+sin(600πn), validando su comportamiento como un SLIT.


Durante la resolución del parcial, utilicé herramientas de inteliencia artificial para aclarar conceptos, obtener apoyo en la estructuración del código, y recibir orientación en la interpretación y análisis de los resultados, asi como también me apoyé de los cuadernos que hay subidos en el github.
